# 生成带噪声的tool标记

In [14]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义类别数
num_classes = 7

# 构建模型：使用预训练的 ResNet50，并将最后一层替换为适合多标签分类的全连接层
model = models.resnet50(pretrained=True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)
model = model.to(device)

# 加载模型权重（请确保路径与实际权重文件一致，这里假设保存为 weight/tool detection.pth）
checkpoint_path = 'weight/tool_detection2.pth'
state_dict = torch.load(checkpoint_path)
model.load_state_dict(state_dict, strict=False)
model.eval()

# 定义测试时的预处理变换（需与训练/测试时的变换保持一致）
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 定义测试数据相关路径（需根据你的文件结构调整）
frame_path = '../autodl-tmp/frames'
test_splite_dir = 'splite/test_videos.txt'

# 加载测试视频列表
with open(test_splite_dir, 'r', encoding='utf-8') as f:
    test_videos = [line.strip() for line in f if line.strip()]

if not test_videos:
    print("测试视频列表为空，请检查文件：", test_splite_dir)
    exit()

# 选择测试视频中的第一个视频
test_video = test_videos[0]
video_frame_dir = os.path.join(frame_path, test_video)
if not os.path.exists(video_frame_dir):
    print(f"视频帧文件夹不存在：{video_frame_dir}")
    exit()

# 获取视频帧列表并按文件名排序，按照数据集构造逻辑跳过第一个图片
img_list = sorted(os.listdir(video_frame_dir))[1:]
if not img_list:
    print(f"在视频文件夹 {video_frame_dir} 中未找到图像文件。")
    exit()

# 选择其中一张图片（这里选取列表中的第一张）
img_path = os.path.join(video_frame_dir, img_list[963])
print("选择的图片路径：", img_path)

# 打开并预处理图像
image = Image.open(img_path).convert("RGB")
image_tensor = test_transform(image).unsqueeze(0)  # 增加 batch 维度

# 模型预测
with torch.no_grad():
    outputs = model(image_tensor.to(device))
    # 使用 sigmoid 激活并以 0.5 为阈值转换为 0/1
    probabilities = torch.sigmoid(outputs)
    predictions = (probabilities > 0.5).int().cpu().numpy()[0]

# 输出预测结果
print(f"\n图片 {img_path} 的预测结果：")
for i, pred in enumerate(predictions):
    print(f"工具 {i+1}: {pred}")


选择的图片路径： ../autodl-tmp/frames/video02/00964.png

图片 ../autodl-tmp/frames/video02/00964.png 的预测结果：
工具 1: 1
工具 2: 0
工具 3: 0
工具 4: 0
工具 5: 0
工具 6: 0
工具 7: 0
